# Test Case LVV-T1802
This case will verify that the integration of the M2 Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s) powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [ ]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

In [ ]:
#For the setCompensateMode tests we need the MTMount and the MTRotator CAVE: here started as controller
mount = salobj.Controller(name="MTMount")
rotator = salobj.Controller(name="MTRotator")
await mount.start_task
await rotator.start_task

In [ ]:
#This sets the rotator angle
# Once we have model coefficients for rotation
# you may want to repeat this for different angles.
# But for now you need only do it once.
rotator.evt_target.set_put(rotation=0)

In [ ]:
# Do this for every desired elevation angle.
mount.evt_target.set_put(elevation=45)

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

In [ ]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [ ]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)

In [ ]:
start_time = datetime.now()
test_message = "M2 Hexapod Integration Test"
#What means index=1 here?
script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

This is how you start the remote for the CSC.

In [ ]:
domain = salobj.Domain()
print (domain)

In [ ]:
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 2
hexapod_csc = salobj.Remote(name="MTHexapod", domain=domain, index=csc_index)

print(hexapod_csc)
await asyncio.sleep(15)
#Bo:Check which hexapod we are controlling
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)

print(hexapod_csc.salinfo.index)

await hexapod_csc.start_task
#This is to bring the state machine in the right starting state
#await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)
#await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

In [ ]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
#may need to wait a few seconds before event shows up in EFD
await asyncio.sleep(15.)
from lsst_efd_client import EfdClient

#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [ ]:
dfe

In [ ]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [ ]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [ ]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.")

    #This is only for the real system. The simulater only has the AVAILABLE offline state 
 
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_start.set_start() 

In [ ]:
await hexapod_csc.cmd_enable.set_start()

In [ ]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
await hexapod_csc.cmd_stop.set_start()

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test move command")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await asyncio.sleep(15.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test move command")

In [ ]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# thermal sensors

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Stop command")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()

await asyncio.sleep(15.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Stop command")

In [ ]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await asyncio.sleep(STD_WAIT)

This block moves the hexapod via the lookup table:

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test setCompensationMode")

CAVE: The setCompensationMode expects infrormation from the MTmount(elevation) and MTRotator(rotation).

To use the setCompensationMode at the summit the controller for MTMount or the MTMount simulater needs to run and the actual MTRotator CSC needs to available at the summit.

To test this at NCSA the simulators must be stopped first.
Use the notebook MTMount-Controller the simulator, start the controller and reverse the steps after finishing testing.

For MTRotator the controller can not be used at NCSA. The simulator is always sending data even in OfflinePublishOnly state. Use the simulator at NCSA and the actual component at the summit

To test the LUT compensation use: hdrass/Camera_Hexapod/hex_diagnostics.ipynb

In [ ]:

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test setCompensationMode")

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test offset command")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test offset command")

In [ ]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test set pivot point")#Pivot test
#Step 29
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


In [ ]:
#Step 30 -- set a new Pivot point
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=0)
await asyncio.sleep(10.)

In [ ]:
#Step 31
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)

In [ ]:
#Set the pivot point back to the original value (0.703m in um)
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=703000)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test set pivot point")

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
#Endurance test for the M2 hexapod. CAVE: This are very long moves. This can overheat the hexapod see.
#LVV-T1600 for shorter moves

script.log.info(f"START- {test_message} -- LVV-T1802 Endurance Test")
for i in range(100):
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


script.log.info(f"STOPP- {test_message} -- LVV-T1802 Endurance Test ")

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection